In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import numpy as np

import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

## Data Loading

In [2]:
import helper
data = helper.load_RWHAR(sel_location = "chest")

from dataset import TimeSeriesDataset
dtset = TimeSeriesDataset(data)

weight = helper.dist(dtset, 8)

train, val = helper.split(dtset, val_pc = 0.3)
train_weight = helper.dist(train, 8)
val_weight = helper.dist(val, 8)
print(train_weight, val_weight)
train_iter = torch.utils.data.DataLoader(train, batch_size = 128, shuffle = True, num_workers = 10, pin_memory = True)
val_iter = torch.utils.data.DataLoader(val, batch_size = 128, num_workers = 10, pin_memory = True)

File exists. Loading
Selecting location :  chest
Windowing
Done!
[[0.07678656]
 [0.1270238 ]
 [0.01995726]
 [0.13473867]
 [0.17751458]
 [0.15476837]
 [0.15513057]
 [0.15408019]] [[0.07860041]
 [0.12119675]
 [0.01834009]
 [0.14173428]
 [0.16979378]
 [0.15449628]
 [0.15821501]
 [0.15762339]]


In [11]:
from DeepConvLSTM_model import DeepConvNet
from Validation_model import Net
net = DeepConvNet(in_channels = 6,input_size = 50, hidden_size = 512, output_size = 8, conv_filter = (3,5), conv_padding = (1,2))
model = Net(model = net, num_classes = 8, classes_weight = torch.tensor(train_weight, dtype = torch.float), lr = 0.001)

trainer = pl.Trainer(gpus=-1,
                     max_epochs=100,
                    #  limit_train_batches=0.1,
                    #  limit_val_batches=0.1,
                     log_every_n_steps = 200,
                     callbacks = [EarlyStopping(monitor = "val_f1_score", patience = 5, mode = "max")],
                     logger = TensorBoardLogger(save_dir = 'Validation_LSTM_logs/', name = "RWHAR"),
                     stochastic_weight_avg=True
                     )
trainer.fit(model, train_iter, val_iter)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | DeepConvNet      | 6.2 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
6.2 M     Trainable params
0         Non-trainable params
6.2 M     Total params
24.934    Total estimated model params size (MB)


1

In [13]:
from TransformerClassifier import TransformerClassifier
from Validation_model import Net
net = TransformerClassifier(in_channels = 6, output_size = 8, d_model = 50, nhead = 5, dim_feedforward = 10000, num_layers = 2)
model = Net(model = net, num_classes = 8, classes_weight = torch.tensor(train_weight, dtype = torch.float), lr = 0.01)

trainer = pl.Trainer(gpus=-1,
                     max_epochs=100,
                    #  limit_train_batches=0.1,
                    #  limit_val_batches=0.1,
                     log_every_n_steps = 200,
                     callbacks = [EarlyStopping(monitor = "val_f1_score", patience = 7, mode = "max")],
                     logger = TensorBoardLogger(save_dir = 'Validation_transformer_logs/', name = "RWHAR"),
                     stochastic_weight_avg=True
                     )
trainer.fit(model, train_iter, val_iter)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name      | Type                  | Params
----------------------------------------------------
0 | model     | TransformerClassifier | 2.1 M 
1 | criterion | CrossEntropyLoss      | 0     
----------------------------------------------------
2.1 M     Trainable params
0         Non-trainable params
2.1 M     Total params
8.234     Total estimated model params size (MB)


1

## Saved model

### Transformer Classifier

Version : 3

location : Validation_transformer_logs/RWHAR/version_3/checkpoints/epoch=66-step=14471.ckpt

F1 score : 88.7%

loss : 0.3502

in_channels = 6, output_size = 8, d_model = 50, nhead = 5, dim_feedforward = 10000, num_layers = 5

### LSTM model

version : 1

location : Validation_LSTM_logs/RWHAR/version_1/checkpoints/epoch=14-step=3239.ckpt

F1 score : 86.22%

loss : 0.3768

in_channels = 6,input_size = 50, hidden_size = 512, output_size = 8, conv_filter = (3,5), conv_padding = (1,2)
